In [4]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [5]:
os.getcwd()

'/Users/caseyh/Desktop/footy/data_import'

In [6]:
main=pd.read_csv('/Users/caseyh/Desktop/footy/player_stats.csv')

In [7]:
main.head()

,Player,round,team,opponent,Disposals,Kicks,Handballs,Goals,Behinds,Marks,...,Goal Assists,Frees,Frees Against,Inside 50s,Rebounds,% Played,Subs,Clangers,One Percenters,Bounces
0,jake bowey,R1,MEL,WBD,17.0,10.0,7.0,0.0,0.0,3.0,...,0.0,1.0,0.0,3.0,3.0,73.0,-,0.0,1.0,0.0
1,angus brayshaw,R1,MEL,WBD,22.0,14.0,8.0,0.0,0.0,10.0,...,0.0,1.0,1.0,2.0,4.0,86.0,-,3.0,0.0,0.0
2,ben brown,R1,MEL,WBD,15.0,12.0,3.0,4.0,1.0,7.0,...,1.0,1.0,0.0,2.0,1.0,81.0,-,2.0,1.0,0.0
3,kade chandler,R1,MEL,WBD,16.0,11.0,5.0,1.0,2.0,8.0,...,2.0,1.0,0.0,4.0,0.0,81.0,-,1.0,0.0,0.0
4,bayley fritsch,R1,MEL,WBD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def rearrange(value):
    return '-'.join(' '.join(value.lower().split(', ')[::-1]).split(' '))

In [9]:
main['Player'] = main['Player'].apply(lambda x: rearrange(x))

In [10]:
mapping = {'CAR':'carlton-blues','MEL':'melbourne-demons','BRL':'brisbane-lions','STK':'st-kilda-saints','COL':'collingwood-magpies','GCS':'gold-coast-suns','GWS':'greater-western-sydney-giants','FRE':'fremantle-dockers','POR':'port-adelaide-power','WCE':'west-coast-eagles','SYD':'sydney-swans','NTH':'kangaroos','HAW':'hawthorn-hawks','GEE':'geelong-cats','RCH':'richmond-tigers','WBD':'western-bulldogs','ESS':'essendon-bombers','ADE':'adelaide-crows'}

In [11]:
for key in mapping.keys():
    main.loc[main['team']==key, 'team'] = mapping[key]

In [12]:
dic = main.groupby(['Player', 'team']).count().reset_index()[['Player', 'team']].to_dict()

In [13]:
dic

{'Player': {0: 'aaron-cadman',
  1: 'aaron-francis',
  2: 'aaron-hall',
  3: 'aaron-naughton',
  4: 'adam-cerra',
  5: 'adam-kennedy',
  6: 'adam-saad',
  7: 'adam-tomlinson',
  8: 'adam-treloar',
  9: 'aidan-corr',
  10: 'aiden-bonar',
  11: 'alex-cincotta',
  12: 'alex-davies',
  13: 'alex-keath',
  14: 'alex-neal-bullen',
  15: 'alex-pearce',
  16: 'alex-sexton',
  17: 'alex-witherden',
  18: 'aliir-aliir',
  19: 'alwyn-davey',
  20: 'andrew-brayshaw',
  21: 'andrew-gaff',
  22: 'andrew-mcgrath',
  23: 'andrew-phillips',
  24: 'angus-brayshaw',
  25: 'angus-sheldrick',
  26: 'anthony-caminiti',
  27: 'anthony-mcdonald-tipungwuti',
  28: 'anthony-scott',
  29: 'archie-perkins',
  30: 'arthur-jones',
  31: 'ash-johnson',
  32: 'bailey-banfield',
  33: 'bailey-dale',
  34: 'bailey-humphrey',
  35: 'bailey-laurie',
  36: 'bailey-macdonald',
  37: 'bailey-scott',
  38: 'bailey-smith',
  39: 'bailey-williams',
  40: 'bailey-williams',
  41: 'bayley-fritsch',
  42: 'beau-mccreery',
  43: '

In [14]:
%time
output = pd.DataFrame({}, columns = ['Player','position','age','games played','dob','origin','height','weight'])
for i in range(len(dic['Player'])):
    try:
        url = f"https://www.footywire.com/afl/footy/pp-{dic['team'][i]}--{dic['Player'][i]}" 
    
        soup = BeautifulSoup(requests.get(url).content, 'html.parser')
        info1 = soup.find_all(id = 'playerProfileData1')[0].text.split()
        info2 = soup.find_all(id = 'playerProfileData2')[0].text.split()

        info = info1 + info2

        lst_index = []
        for j, value in enumerate(info):
            if value in ['Age:', 'Games:', 'Born:', 'Origin:', 'Height:', 'Weight:','Position:']:
                lst_index.append(j)

        info_list = []
        for k in range(len(lst_index) - 1):
            a = lst_index[k]
            b = lst_index[k+1]
            info_list.append(info[a + 1:b])
        info_list.append(info[lst_index[-1] + 1:])

        print(i)
        print(' '.join(dic['Player'][i].split('-')))
        print(info_list)

        age = ' '.join(info_list[0])
        games = ' '.join(info_list[1])
        birth = ' '.join(info_list[2])
        place = ' '.join(info_list[3])
        height = ' '.join(info_list[4])
        weight = ' '.join(info_list[5])
        position = ' '.join(info_list[6])



        new_row = {'Player': ' '.join(dic['Player'][i].split('-')), 'position': position, 'age': age, 'games played':games, 
              'dob': birth, 'origin': place, 'height':height, 'weight':weight}

        output.loc[len(output)] = new_row
    except:
        continue


CPU times: user 7 µs, sys: 31 µs, total: 38 µs
Wall time: 14.1 µs
0
aaron cadman
[['19yr', '3mth'], ['9'], ['March', '3,', '2004'], ['Greater', 'Western', 'Victoria', 'Rebels'], ['195cm'], ['92kg'], ['Forward']]
1
aaron francis
[['25yr', '10mth'], ['63'], ['August', '10,', '1997'], ['West', 'Adelaide'], ['193cm'], ['92kg'], ['Defender,', 'Forward']]
2
aaron hall
[['32yr', '7mth'], ['161'], ['November', '9,', '1990'], ['Hobart'], ['185cm'], ['85kg'], ['Defender']]
3
aaron naughton
[['23yr', '6mth'], ['114'], ['November', '30,', '1999'], ['Peel', 'Thunder'], ['195cm'], ['93kg'], ['Forward']]
4
adam cerra
[['23yr', '8mth'], ['108'], ['October', '7,', '1999'], ['Eastern', 'Ranges'], ['187cm'], ['86kg'], ['Midfield']]
5
adam kennedy
[['30yr', '11mth'], ['153'], ['July', '12,', '1992'], ['Western', 'Jets'], ['184cm'], ['80kg'], ['Defender,', 'Forward']]
6
adam saad
[['28yr', '11mth'], ['165'], ['July', '23,', '1994'], ['Coburg'], ['178cm'], ['77kg'], ['Defender']]
7
adam tomlinson
[['29yr', 

65
brandon ellis
[['29yr', '10mth'], ['238'], ['August', '3,', '1993'], ['Calder', 'Cannons'], ['181cm'], ['82kg'], ['Defender,', 'Midfield']]
66
brandon starcevich
[['23yr', '11mth'], ['85'], ['July', '24,', '1999'], ['East', 'Perth'], ['187cm'], ['92kg'], ['Defender']]
67
brandon walker
[['20yr', '8mth'], ['42'], ['October', '17,', '2002'], ['East', 'Fremantle'], ['186cm'], ['80kg'], ['Defender']]
68
brandon zerk thatcher
[['24yr', '10mth'], ['42'], ['August', '25,', '1998'], ['Sturt'], ['195cm'], ['89kg'], ['Defender']]
69
brayden fiorini
[['25yr', '10mth'], ['88'], ['August', '22,', '1997'], ['Northern', 'Knights'], ['186cm'], ['84kg'], ['Midfield']]
70
brayden maynard
[['26yr', '9mth'], ['175'], ['September', '20,', '1996'], ['Sandringham', 'Dragons'], ['189cm'], ['93kg'], ['Defender']]
71
brennan cox
[['24yr', '10mth'], ['96'], ['August', '13,', '1998'], ['Woodville-West', 'Torrens'], ['195cm'], ['100kg'], ['Defender']]
72
brent daniels
[['24yr', '3mth'], ['73'], ['March', '9,', 

131
daniel turner
[['21yr', '4mth'], ['2'], ['January', '28,', '2002'], ['Murray', 'Bushrangers'], ['194cm'], ['88kg'], ['Defender']]
132
darcy byrne jones
[['27yr', '9mth'], ['163'], ['September', '20,', '1995'], ['Oakleigh', 'Chargers'], ['181cm'], ['76kg'], ['Defender,', 'Forward']]
133
darcy cameron
[['27yr', '11mth'], ['61'], ['July', '18,', '1995'], ['Claremont'], ['204cm'], ['101kg'], ['Forward,', 'Ruck']]
134
darcy fogarty
[['23yr', '9mth'], ['70'], ['September', '25,', '1999'], ['Glenelg'], ['194cm'], ['99kg'], ['Forward']]
135
darcy fort
[['29yr', '10mth'], ['31'], ['August', '6,', '1993'], ['Central', 'District'], ['204cm'], ['98kg'], ['Forward,', 'Ruck']]
136
darcy gardiner
[['27yr', '9mth'], ['156'], ['September', '22,', '1995'], ['Geelong', 'Falcons'], ['192cm'], ['91kg'], ['Defender']]
137
darcy macpherson
[['25yr', '7mth'], ['86'], ['October', '29,', '1997'], ['Northern', 'Knights'], ['174cm'], ['79kg'], ['Defender,', 'Forward']]
138
darcy moore
[['27yr', '5mth'], ['140

199
hewago oea
[['21yr', '7mth'], ['9'], ['November', '13,', '2001'], ['Suns', 'Academy'], ['172cm'], ['70kg'], ['Forward']]
200
hugh greenwood
[['31yr', '3mth'], ['114'], ['March', '6,', '1992'], ['Perth', 'Wildcats'], ['191cm'], ['94kg'], ['Midfield']]
201
hugh mccluggage
[['25yr', '3mth'], ['143'], ['March', '3,', '1998'], ['North', 'Ballarat', 'Rebels'], ['185cm'], ['86kg'], ['Midfield']]
202
hugo ralphsmith
[['21yr', '7mth'], ['30'], ['November', '9,', '2001'], ['Sandringham', 'Dragons'], ['188cm'], ['83kg'], ['Defender']]
203
hunter clark
[['24yr', '2mth'], ['79'], ['March', '26,', '1999'], ['Dandenong', 'Stingrays'], ['186cm'], ['80kg'], ['Defender,', 'Midfield']]
204
isaac cumming
[['24yr', '10mth'], ['63'], ['August', '11,', '1998'], ['Broken', 'Hill'], ['188cm'], ['84kg'], ['Defender']]
205
isaac heeney
[['27yr', '1mth'], ['167'], ['May', '5,', '1996'], ['Cardiff'], ['185cm'], ['87kg'], ['Forward']]
206
isaac quaynor
[['23yr', '5mth'], ['73'], ['January', '15,', '2000'], ['Oa

265
james peatling
[['22yr', '10mth'], ['26'], ['August', '21,', '2000'], ['GWS', 'Giants'], ['185cm'], ['78kg'], ['Forward']]
266
james rowbottom
[['22yr', '9mth'], ['83'], ['September', '19,', '2000'], ['Oakleigh', 'Chargers'], ['186cm'], ['82kg'], ['Midfield']]
267
james sicily
[['28yr', '5mth'], ['127'], ['January', '6,', '1995'], ['Western', 'Jets'], ['187cm'], ['87kg'], ['Defender']]
268
james tsitas
[['28yr', '3mth'], ['3'], ['March', '3,', '1995'], ['Geelong', 'Falcons'], ['181cm'], ['84kg'], ['Midfield,', 'Forward']]
269
james tunstill
[['19yr', '11mth'], ['4'], ['July', '18,', '2003'], ['East', 'Perth'], ['187cm'], ['75kg'], ['Midfield']]
270
james worpel
[['24yr', '5mth'], ['93'], ['January', '24,', '1999'], ['Geelong', 'Falcons'], ['186cm'], ['88kg'], ['Midfield']]
271
jamie cripps
[['31yr', '2mth'], ['219'], ['April', '23,', '1992'], ['East', 'Fremantle'], ['183cm'], ['83kg'], ['Forward']]
272
jamie elliott
[['February', '6,', '1973', 'Playing'], ['185cm', 'Playing'], ['85

332
jye caldwell
[['22yr', '8mth'], ['48'], ['September', '28,', '2000'], ['Bendigo', 'Pioneers'], ['183cm'], ['83kg'], ['Midfield,', 'Forward']]
333
jye menzie
[['20yr', '7mth'], ['15'], ['October', '28,', '2002'], ['South', 'Adelaide'], ['180cm'], ['81kg'], ['Forward']]
334
kade chandler
[['23yr', '5mth'], ['24'], ['January', '13,', '2000'], ['Norwood'], ['175cm'], ['80kg'], ['Forward']]
335
kai lohmann
[['20yr', '1mth'], ['4'], ['May', '6,', '2003'], ['GWV', 'Rebels'], ['185cm'], ['77kg'], ['Forward']]
336
kaine baldwin
[['21yr'], ['6'], ['May', '30,', '2002'], ['Glenelg'], ['193cm'], ['95kg'], ['Forward']]
337
kamdyn mcintosh
[['29yr', '2mth'], ['160'], ['April', '3,', '1994'], ['Peel', 'Thunder'], ['191cm'], ['92kg'], ['Midfield']]
338
kane farrell
[['24yr', '3mth'], ['61'], ['March', '17,', '1999'], ['Bendigo', 'Pioneers'], ['182cm'], ['78kg'], ['Defender,', 'Midfield']]
339
karl amon
[['27yr', '10mth'], ['138'], ['August', '19,', '1995'], ['Sandringham', 'Dragons'], ['181cm'], [

410
matthew kennedy
[['February', '4,', '1970', 'Playing'], ['191cm', 'Playing'], ['98kg']]
412
matthew taberner
[['30yr'], ['120'], ['June', '17,', '1993'], ['Murray', 'Bushrangers'], ['198cm'], ['99kg'], ['Forward']]
413
maurice rioli
[['September', '1,', '1957', 'Playing'], ['175cm', 'Playing'], ['76kg']]
414
max gawn
[['31yr', '5mth'], ['192'], ['December', '30,', '1991'], ['Sandringham', 'Dragons'], ['209cm'], ['111kg'], ['Ruck']]
415
max holmes
[['20yr', '9mth'], ['42'], ['August', '29,', '2002'], ['Sandringham', 'Dragons'], ['189cm'], ['83kg'], ['Midfield']]
416
max king
[['September', '24,', '1995'], ['Murray', 'Bushrangers', 'Playing'], ['200cm', 'Playing'], ['95kg'], ['Forward,', 'Ruck']]
417
max lynch
[['24yr', '9mth'], ['11'], ['September', '12,', '1998'], ['Murray', 'Bushrangers'], ['200cm'], ['101kg'], ['Ruck']]
418
max michalanney
[['19yr', '3mth'], ['14'], ['February', '26,', '2004'], ['Norwood'], ['190cm'], ['78kg'], ['Defender']]
419
max ramsden
[['20yr', '2mth'], ['1

490
rhys stanley
[['32yr', '6mth'], ['188'], ['December', '1,', '1990'], ['West', 'Adelaide'], ['200cm'], ['102kg'], ['Ruck']]
491
riley bonner
[['26yr', '3mth'], ['91'], ['March', '7,', '1997'], ['West', 'Adelaide'], ['190cm'], ['85kg'], ['Defender,', 'Midfield']]
492
riley garcia
[['22yr', '4mth'], ['18'], ['January', '30,', '2001'], ['Swan', 'Districts'], ['177cm'], ['77kg'], ['Forward']]
493
riley thilthorpe
[['20yr', '11mth'], ['37'], ['July', '7,', '2002'], ['West', 'Adelaide', 'Fc'], ['202cm'], ['105kg'], ['Forward']]
494
robbie fox
[['30yr', '2mth'], ['77'], ['April', '16,', '1993'], ['Coburg'], ['185cm'], ['84kg'], ['Defender']]
495
robbie mccomb
[['27yr', '6mth'], ['18'], ['December', '19,', '1995'], ['Footscray'], ['181cm'], ['82kg'], ['Midfield,', 'Forward']]
496
rory atkins
[['28yr', '11mth'], ['126'], ['July', '12,', '1994'], ['Calder', 'Cannons'], ['186cm'], ['85kg'], ['Defender,', 'Midfield']]
497
rory laird
[['29yr', '5mth'], ['216'], ['December', '29,', '1993'], ['Wes

556
todd goldstein
[['34yr', '11mth'], ['308'], ['July', '1,', '1988'], ['Oakleigh'], ['201cm'], ['103kg'], ['Ruck']]
557
todd marshall
[['24yr', '8mth'], ['87'], ['October', '8,', '1998'], ['Murray', 'Bushrangers'], ['198cm'], ['92kg'], ['Forward']]
558
tom atkins
[['27yr', '9mth'], ['96'], ['September', '18,', '1995'], ['Geelong'], ['180cm'], ['82kg'], ['Defender,', 'Midfield']]
559
tom barrass
[['27yr', '8mth'], ['129'], ['October', '8,', '1995'], ['Claremont'], ['197cm'], ['96kg'], ['Defender']]
561
tom clurey
[['29yr', '3mth'], ['124'], ['March', '23,', '1994'], ['Murray', 'Bushrangers'], ['193cm'], ['95kg'], ['Defender']]
562
tom cole
[['26yr'], ['82'], ['May', '28,', '1997'], ['Sandhurst', 'Football', 'Netball', 'Club'], ['188cm'], ['83kg'], ['Defender']]
563
tom de koning
[['23yr', '11mth'], ['50'], ['July', '16,', '1999'], ['Dandenong', 'Stingrays'], ['203cm'], ['101kg'], ['Ruck']]
564
tom doedee
[['26yr', '3mth'], ['82'], ['March', '1,', '1997'], ['Geelong', 'Falcons'], ['187

In [15]:
output

,Player,position,age,games played,dob,origin,height,weight
0,aaron cadman,Forward,19yr 3mth,9,"March 3, 2004",Greater Western Victoria Rebels,195cm,92kg
1,aaron francis,"Defender, Forward",25yr 10mth,63,"August 10, 1997",West Adelaide,193cm,92kg
2,aaron hall,Defender,32yr 7mth,161,"November 9, 1990",Hobart,185cm,85kg
3,aaron naughton,Forward,23yr 6mth,114,"November 30, 1999",Peel Thunder,195cm,93kg
4,adam cerra,Midfield,23yr 8mth,108,"October 7, 1999",Eastern Ranges,187cm,86kg
...,...,...,...,...,...,...,...,...
543,zach guthrie,Defender,24yr 11mth,68,"June 30, 1998",Calder Cannons,187cm,84kg
544,zach tuohy,"Defender, Midfield",33yr 6mth,263,"December 10, 1989","County Laois, Ireland",187cm,91kg
545,zaine cordy,"Defender, Forward",26yr 7mth,125,"October 27, 1996",Geelong Falcons,195cm,92kg
546,zak butters,"Midfield, Forward",22yr 9mth,82,"September 8, 2000",Western Jets,181cm,78kg


In [17]:
output.to_csv('/Users/caseyh/Desktop/footy/playerinfo.csv', index = False)